In [111]:
import pandas as pd

# Load dataset
df = pd.read_csv('/content/train.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


## a) **handle missing values**

In [112]:
 # Step 1: Find columns with missing values
missing = df.isnull().sum()
missing = missing[missing > 0]  # Only show columns with missing values

# Step 2: Impute or drop based on missing percentage and type
for col in missing.index:
    missing_ratio = missing[col] / len(df)

    if missing_ratio > 0.85:
        df.drop(columns=[col], inplace=True)
    else:
        if df[col].dtype == 'object':
            mode_val = df[col].mode()[0]
            df[col] = df[col].fillna(mode_val)
        else:
            skewness = df[col].skew()
            if abs(skewness) > 1:
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
            else:
                mean_val = df[col].mean()
                df[col] = df[col].fillna(mean_val)

# Step 3: Display the cleaned dataset in proper tabular format (first 5 rows)
print(" Cleaned Dataset Preview (First 5 Rows):\n")
print(df.head().to_string(index=False))


 Cleaned Dataset Preview (First 5 Rows):

 Unnamed: 0                             Name   Location  Year  Kilometers_Driven Fuel_Type Transmission Owner_Type    Mileage  Engine     Power  Seats  Price
          1 Hyundai Creta 1.6 CRDi SX Option       Pune  2015              41000    Diesel       Manual      First 19.67 kmpl 1582 CC 126.2 bhp    5.0  12.50
          2                     Honda Jazz V    Chennai  2011              46000    Petrol       Manual      First   13 km/kg 1199 CC  88.7 bhp    5.0   4.50
          3                Maruti Ertiga VDI    Chennai  2012              87000    Diesel       Manual      First 20.77 kmpl 1248 CC 88.76 bhp    7.0   6.00
          4  Audi A4 New 2.0 TDI Multitronic Coimbatore  2013              40670    Diesel    Automatic     Second  15.2 kmpl 1968 CC 140.8 bhp    5.0  17.74
          6           Nissan Micra Diesel XV     Jaipur  2013              86999    Diesel       Manual      First 23.08 kmpl 1461 CC  63.1 bhp    5.0   3.50


**justification :**
Using an approach based on data type and missing %, missing values were addressed in order to clean the dataset.  To preserve data integrity, columns that had more than 85% missing data were removed.  To preserve the most prevalent category, missing values for categorical columns like Owner_Type were filled in using the mode.  The distribution of numerical columns, such as Seats, Power, and Mileage, was taken into consideration.  To reduce the impact of outliers, the median was employed when the column was heavily skewed; for normally distributed data, the mean was utilized.  For readability and practicality, the original structure was maintained, including units like "kmpl," "CC," and "bhp."  This methodology guaranteed a comprehensive, dependable, and comprehensible dataset for additional examination.

#**b)remove units from attributes**

In [113]:
#Remove units and convert to float
df['Mileage'] = df['Mileage'].astype(str).str.extract('([\d.]+)').astype(float)
df['Engine'] = df['Engine'].astype(str).str.extract('([\d.]+)').astype(float)
df['Power'] = df['Power'].astype(str).str.extract('([\d.]+)').astype(float)

# Handle 'New_Price' only if it exists (and wasn't dropped)
if 'New_Price' in df.columns:
    df['New_Price'] = df['New_Price'].astype(str).str.replace(' Lakh', '', regex=False)
    df['New_Price'] = pd.to_numeric(df['New_Price'], errors='coerce')

#Display cleaned output (first 5 rows)
print("\n Final Cleaned Dataset (First 5 Rows):\n")
print(df.head().to_string(index=False, float_format='{:,.2f}'.format))



 Final Cleaned Dataset (First 5 Rows):

 Unnamed: 0                             Name   Location  Year  Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage   Engine  Power  Seats  Price
          1 Hyundai Creta 1.6 CRDi SX Option       Pune  2015              41000    Diesel       Manual      First    19.67 1,582.00 126.20   5.00  12.50
          2                     Honda Jazz V    Chennai  2011              46000    Petrol       Manual      First    13.00 1,199.00  88.70   5.00   4.50
          3                Maruti Ertiga VDI    Chennai  2012              87000    Diesel       Manual      First    20.77 1,248.00  88.76   7.00   6.00
          4  Audi A4 New 2.0 TDI Multitronic Coimbatore  2013              40670    Diesel    Automatic     Second    15.20 1,968.00 140.80   5.00  17.74
          6           Nissan Micra Diesel XV     Jaipur  2013              86999    Diesel       Manual      First    23.08 1,461.00  63.10   5.00   3.50


#**c)one hot encode categorical variables**

In [114]:
# Step 1: One-hot encode if needed
columns_to_encode = [col for col in ['Fuel_Type', 'Transmission'] if col in df.columns]

if columns_to_encode:
    df = pd.get_dummies(df, columns=columns_to_encode, drop_first=True)
    print("One-hot encoding applied for:", columns_to_encode)
else:
    print("Columns already encoded or do not exist:", ['Fuel_Type', 'Transmission'])

# Step 2: Get encoded column names
encoded_cols = [col for col in df.columns if 'Fuel_Type_' in col or 'Transmission_' in col]

# Step 3: Format and display output like your sample
print("\nOne-Hot Encoded Output in Table Format:\n")
print(df[encoded_cols].head(5).astype(int).to_string(index=False))


One-hot encoding applied for: ['Fuel_Type', 'Transmission']

One-Hot Encoded Output in Table Format:

 Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Manual
                  0                 0                    1
                  0                 1                    1
                  0                 0                    1
                  0                 0                    0
                  0                 0                    1


#**d)newfeature car-age**

In [115]:
from datetime import datetime

# Create 'Car_Age' by subtracting the manufacturing year from the current year
current_year = datetime.now().year
df['Car_Age'] = current_year - df['Year']

# Display: Confirm new column and show sample values
print(" rows with 'Year' and calculated 'Car_Age':")
print(df[['Year', 'Car_Age']].head())


 rows with 'Year' and calculated 'Car_Age':
   Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


#**e)data operations**

In [116]:
# 1. Select & copy subset of columns
selected_df = df[['Location', 'Year', 'Kilometers_Driven', 'Mileage', 'Engine',
                  'Power', 'Seats', 'Car_Age', 'Price']].copy()

# 2. Filter cars: High mileage (>20) and low price (<5)
filtered_df = selected_df[(selected_df['Mileage'] > 20) & (selected_df['Price'] < 5)]

# 3. Rename column safely
selected_df.rename(columns={'Kilometers_Driven': 'KMs_Driven'}, inplace=True)

# 4. Arrange by descending price
arranged_df = selected_df.sort_values(by='Price', ascending=False)

# 5. Summarize: Mean Price and Mileage by Location
summary_df = selected_df.groupby('Location').agg({
    'Price': 'mean',
    'Mileage': 'mean'
}).reset_index()

# === Display or export results ===

print("Filtered Data:")
print(filtered_df.head())

print("\nTop 5 Expensive Cars:")
print(arranged_df.head())

print("\nSummary by Location:")
print(summary_df.head())


Filtered Data:
     Location  Year  Kilometers_Driven  Mileage  Engine  Power  Seats  \
4      Jaipur  2013              86999    23.08  1461.0   63.1    5.0   
7     Chennai  2012              65932    22.30  1248.0   74.0    5.0   
25  Hyderabad  2013              54000    20.92   998.0   67.1    5.0   
39      Kochi  2014              19698    25.40   624.0   37.5    4.0   
46     Jaipur  2014              75000    24.40  1120.0   71.0    5.0   

    Car_Age  Price  
4        12   3.50  
7        13   1.95  
25       12   2.75  
39       11   1.53  
46       11   3.85  

Top 5 Expensive Cars:
        Location  Year  KMs_Driven  Mileage  Engine  Power  Seats  Car_Age  \
3952   Hyderabad  2017       25000    13.33  2993.0  255.0    5.0        8   
5620       Delhi  2011        6500     6.40  5204.0  560.0    2.0       14   
5752   Hyderabad  2015        8000    12.50  5000.0  488.1    2.0       10   
1457       Kochi  2019       26013    12.65  2993.0  255.0    5.0        6   
1917  C